In [3]:
import numpy as np
import cv2
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from utils import load_data, split_dataset, IMG_DIM
from visualisation import show_samples

In [10]:
class DeskewTransformer(BaseEstimator, TransformerMixin): 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        size = X.shape[-1]
        
        def deskew(img):
            m = cv2.moments(img)
            if abs(m['mu02']) < 1e-2:
                return img.copy()
            skew = m['mu11'] / m['mu02']
            M = np.float32([[1, skew, -0.5 * size * skew], [0, 1, 0]])
            img = cv2.warpAffine(img, 
                                 M, 
                                 (size, size), 
                                 flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
            return img

        return np.array([deskew(img) for img in X])
    
    
class HogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 window_size, 
                 cell_size, 
                 block_size, 
                 block_stride, 
                 n_bins=9):
        self.window_size = window_size
        self.cell_size = cell_size
        self.block_size = block_size
        self.block_stride = block_stride
        self.n_bins = n_bins
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        hog = cv2.HOGDescriptor(self.window_size, 
                                self.block_size, 
                                self.block_stride, 
                                self.cell_size, 
                                self.n_bins)
        return np.array([hog.compute(img).flatten() for img in X])

In [18]:
pipeline = Pipeline([
    ('deskew', DeskewTransformer()),
    ('hog', HogTransformer((IMG_DIM, IMG_DIM), (8, 8), (8, 8), (4, 4))),
    ('clf', DecisionTreeClassifier())
])

parameters = [
    {
        'hog__window_size': ((IMG_DIM, IMG_DIM),),
        'hog__cell_size': ((8, 8),),
        'hog__block_size': ((16, 16),),
        'hog__block_stride': ((8, 8),),
        'clf': (DecisionTreeClassifier(),),
        'clf__max_depth': (5, 10)
    },
    {
        'hog__window_size': ((IMG_DIM, IMG_DIM),),
        'hog__cell_size': ((8, 8),),
        'hog__block_size': ((16, 16),),
        'hog__block_stride': ((8, 8),),
        'clf': (LinearSVC(),),
        'clf__C': (0.01, 0.5)
    }
]

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)

In [19]:
x, y = load_data(DIR)
x_train, x_test, y_train, y_test = split_dataset(x, y)
grid_search.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('deskew', DeskewTransformer()),
                                       ('hog',
                                        HogTransformer(block_size=(8, 8),
                                                       block_stride=(4, 4),
                                                       cell_size=(8, 8),
                                                       n_bins=None,
                                                       window_size=(56, 56))),
                                       ('clf',
                                        DecisionTreeClassifier(ccp_alpha=0.0,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                            